In [2]:
import argparse
import os
import pprint

import gym
import numpy as np
import torch
from torch.utils.tensorboard import SummaryWriter

from tianshou.data import Collector, PrioritizedVectorReplayBuffer, VectorReplayBuffer
from tianshou.env import DummyVectorEnv
from tianshou.policy import DQNPolicy
from tianshou.trainer import offpolicy_trainer
from tianshou.utils import TensorboardLogger
from tianshou.utils.net.common import Net

In [1]:
import gym
import tianshou as ts

env = gym.make('CartPole-v0')

In [3]:
def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--task', type=str, default='CartPole-v0')
    parser.add_argument('--seed', type=int, default=1626)
    parser.add_argument('--eps-test', type=float, default=0.05)
    parser.add_argument('--eps-train', type=float, default=0.1)
    parser.add_argument('--buffer-size', type=int, default=20000)
    parser.add_argument('--lr', type=float, default=1e-3)
    parser.add_argument('--gamma', type=float, default=0.9)
    parser.add_argument('--n-step', type=int, default=3)
    parser.add_argument('--target-update-freq', type=int, default=320)
    parser.add_argument('--epoch', type=int, default=20)
    parser.add_argument('--step-per-epoch', type=int, default=10000)
    parser.add_argument('--step-per-collect', type=int, default=10)
    parser.add_argument('--update-per-step', type=float, default=0.1)
    parser.add_argument('--batch-size', type=int, default=64)
    parser.add_argument(
        '--hidden-sizes', type=int, nargs='*', default=[128, 128, 128, 128]
    )
    parser.add_argument('--training-num', type=int, default=10)
    parser.add_argument('--test-num', type=int, default=100)
    parser.add_argument('--logdir', type=str, default='log')
    parser.add_argument('--render', type=float, default=0.)
    parser.add_argument('--prioritized-replay', action="store_true", default=False)
    parser.add_argument('--alpha', type=float, default=0.6)
    parser.add_argument('--beta', type=float, default=0.4)
    parser.add_argument(
        '--device', type=str, default='cuda' if torch.cuda.is_available() else 'cpu'
    )
    args = parser.parse_known_args()[0]
    return args

In [4]:

def test_dqn(args=get_args()):
    env = gym.make(args.task)
    args.state_shape = env.observation_space.shape or env.observation_space.n
    args.action_shape = env.action_space.shape or env.action_space.n
    # train_envs = gym.make(args.task)
    # you can also use tianshou.env.SubprocVectorEnv
    train_envs = DummyVectorEnv(
        [lambda: gym.make(args.task) for _ in range(args.training_num)]
    )
    # test_envs = gym.make(args.task)
    test_envs = DummyVectorEnv(
        [lambda: gym.make(args.task) for _ in range(args.test_num)]
    )
    # seed
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    train_envs.seed(args.seed)
    test_envs.seed(args.seed)
    # Q_param = V_param = {"hidden_sizes": [128]}
    # model
    net = Net(
        args.state_shape,
        args.action_shape,
        hidden_sizes=args.hidden_sizes,
        device=args.device,
        # dueling=(Q_param, V_param),
    ).to(args.device)
    optim = torch.optim.Adam(net.parameters(), lr=args.lr)
    policy = DQNPolicy(
        net,
        optim,
        args.gamma,
        args.n_step,
        target_update_freq=args.target_update_freq,
    )
    # buffer
    if args.prioritized_replay:
        buf = PrioritizedVectorReplayBuffer(
            args.buffer_size,
            buffer_num=len(train_envs),
            alpha=args.alpha,
            beta=args.beta,
        )
    else:
        buf = VectorReplayBuffer(args.buffer_size, buffer_num=len(train_envs))
    # collector
    train_collector = Collector(policy, train_envs, buf, exploration_noise=True)
    test_collector = Collector(policy, test_envs, exploration_noise=True)
    # policy.set_eps(1)
    train_collector.collect(n_step=args.batch_size * args.training_num)
    # log
    log_path = os.path.join(args.logdir, args.task, 'dqn')
    writer = SummaryWriter(log_path)
    logger = TensorboardLogger(writer)

    def save_fn(policy):
        torch.save(policy.state_dict(), os.path.join(log_path, 'policy.pth'))

    def stop_fn(mean_rewards):
        return mean_rewards >= env.spec.reward_threshold

    def train_fn(epoch, env_step):
        # eps annnealing, just a demo
        if env_step <= 10000:
            policy.set_eps(args.eps_train)
        elif env_step <= 50000:
            eps = args.eps_train - (env_step - 10000) / \
                40000 * (0.9 * args.eps_train)
            policy.set_eps(eps)
        else:
            policy.set_eps(0.1 * args.eps_train)

    def test_fn(epoch, env_step):
        policy.set_eps(args.eps_test)

    # trainer
    result = offpolicy_trainer(
        policy,
        train_collector,
        test_collector,
        args.epoch,
        args.step_per_epoch,
        args.step_per_collect,
        args.test_num,
        args.batch_size,
        update_per_step=args.update_per_step,
        train_fn=train_fn,
        test_fn=test_fn,
        stop_fn=stop_fn,
        save_fn=save_fn,
        logger=logger,
    )
    assert stop_fn(result['best_reward'])

    if __name__ == '__main__':
        pprint.pprint(result)
        # Let's watch its performance!
        env = gym.make(args.task)
        policy.eval()
        policy.set_eps(args.eps_test)
        collector = Collector(policy, env)
        result = collector.collect(n_episode=1, render=args.render)
        rews, lens = result["rews"], result["lens"]
        print(f"Final reward: {rews.mean()}, length: {lens.mean()}")


In [5]:
def test_pdqn(args=get_args()):
    args.prioritized_replay = True
    args.gamma = .95
    args.seed = 1
    test_dqn(args)

In [6]:
test_dqn(get_args())

Epoch #1:  95%|#########5| 9530/10000 [00:14<00:00, 678.88it/s, env_step=9530, len=200, n/ep=1, n/st=10, rew=200.00]             

{'best_result': '199.74 ± 1.87',
 'best_reward': 199.74,
 'duration': '14.15s',
 'test_episode': 600,
 'test_speed': '18801.27 step/s',
 'test_step': 85235,
 'test_time': '4.53s',
 'train_episode': 493,
 'train_speed': '990.82 step/s',
 'train_step': 9530,
 'train_time/collector': '1.98s',
 'train_time/model': '7.63s'}
Final reward: 200.0, length: 200.0



C:\Users\Lenovo\anaconda3\lib\site-packages\tianshou\data\collector.py:61: UserWarning: Single environment detected, wrap to DummyVectorEnv.
  warnings.warn("Single environment detected, wrap to DummyVectorEnv.")


In [8]:
# policy.load_state_dict(torch.load('log/dqn.pth'))
test_pdqn(args=get_args())

Epoch #1: 10001it [00:08, 1149.01it/s, env_step=10000, len=200, loss=0.320, n/ep=0, n/st=10, rew=200.00]                          
Epoch #2:   3%|2         | 270/10000 [00:00<00:06, 1477.77it/s, env_step=10270, len=153, loss=0.379, n/ep=0, n/st=10, rew=153.00]

Epoch #1: test_reward: 193.060000 ± 12.457785, best_reward: 193.060000 ± 12.457785 in #1


Epoch #2:  76%|#######6  | 7630/10000 [00:07<00:02, 1041.37it/s, env_step=17630, len=200, n/ep=1, n/st=10, rew=200.00]            

{'best_result': '197.46 ± 6.05',
 'best_reward': 197.46,
 'duration': '16.89s',
 'test_episode': 600,
 'test_speed': '22325.26 step/s',
 'test_step': 82848,
 'test_time': '3.71s',
 'train_episode': 433,
 'train_speed': '1337.39 step/s',
 'train_step': 17630,
 'train_time/collector': '2.87s',
 'train_time/model': '10.31s'}
Final reward: 200.0, length: 200.0



C:\Users\Lenovo\anaconda3\lib\site-packages\tianshou\data\collector.py:61: UserWarning: Single environment detected, wrap to DummyVectorEnv.
  warnings.warn("Single environment detected, wrap to DummyVectorEnv.")


In [9]:
args = get_args()
env = gym.make(args.task)
args.state_shape = env.observation_space.shape or env.observation_space.n
args.action_shape = env.action_space.shape or env.action_space.n
# train_envs = gym.make(args.task)
# you can also use tianshou.env.SubprocVectorEnv
train_envs = DummyVectorEnv(
    [lambda: gym.make(args.task) for _ in range(args.training_num)]
)
# test_envs = gym.make(args.task)
test_envs = DummyVectorEnv(
    [lambda: gym.make(args.task) for _ in range(args.test_num)]
)
# seed
np.random.seed(args.seed)
torch.manual_seed(args.seed)
train_envs.seed(args.seed)
test_envs.seed(args.seed)
# Q_param = V_param = {"hidden_sizes": [128]}
# model
net = Net(
    args.state_shape,
    args.action_shape,
    hidden_sizes=args.hidden_sizes,
    device=args.device,
    # dueling=(Q_param, V_param),
).to(args.device)
optim = torch.optim.Adam(net.parameters(), lr=args.lr)
policy = DQNPolicy(
    net,
    optim,
    args.gamma,
    args.n_step,
    target_update_freq=args.target_update_freq,
)
# buffer
if args.prioritized_replay:
    buf = PrioritizedVectorReplayBuffer(
        args.buffer_size,
        buffer_num=len(train_envs),
        alpha=args.alpha,
        beta=args.beta,
    )
else:
    buf = VectorReplayBuffer(args.buffer_size, buffer_num=len(train_envs))
# collector
train_collector = Collector(policy, train_envs, buf, exploration_noise=True)
test_collector = Collector(policy, test_envs, exploration_noise=True)
# policy.set_eps(1)
train_collector.collect(n_step=args.batch_size * args.training_num)
# log
log_path = os.path.join(args.logdir, args.task, 'dqn')
writer = SummaryWriter(log_path)
logger = TensorboardLogger(writer)

def save_fn(policy):
    torch.save(policy.state_dict(), os.path.join(log_path, 'policy.pth'))

def stop_fn(mean_rewards):
    return mean_rewards >= env.spec.reward_threshold

def train_fn(epoch, env_step):
    # eps annnealing, just a demo
    if env_step <= 10000:
        policy.set_eps(args.eps_train)
    elif env_step <= 50000:
        eps = args.eps_train - (env_step - 10000) / \
            40000 * (0.9 * args.eps_train)
        policy.set_eps(eps)
    else:
        policy.set_eps(0.1 * args.eps_train)

def test_fn(epoch, env_step):
    policy.set_eps(args.eps_test)

# trainer
result = offpolicy_trainer(
    policy,
    train_collector,
    test_collector,
    args.epoch,
    args.step_per_epoch,
    args.step_per_collect,
    args.test_num,
    args.batch_size,
    update_per_step=args.update_per_step,
    train_fn=train_fn,
    test_fn=test_fn,
    stop_fn=stop_fn,
    save_fn=save_fn,
    logger=logger,
)
assert stop_fn(result['best_reward'])

Epoch #1:  95%|#########5| 9530/10000 [00:10<00:00, 936.59it/s, env_step=9530, len=200, n/ep=1, n/st=10, rew=200.00]             


In [10]:
pprint.pprint(result)
# Let's watch its performance!
env = gym.make(args.task)
policy.eval()
policy.set_eps(args.eps_test)
collector = Collector(policy, env)
result = collector.collect(n_episode=1, render=args.render)
rews, lens = result["rews"], result["lens"]
print(f"Final reward: {rews.mean()}, length: {lens.mean()}")

{'best_result': '199.74 ± 1.87',
 'best_reward': 199.74,
 'duration': '10.22s',
 'test_episode': 600,
 'test_speed': '24355.80 step/s',
 'test_step': 85235,
 'test_time': '3.50s',
 'train_episode': 493,
 'train_speed': '1417.27 step/s',
 'train_step': 9530,
 'train_time/collector': '1.49s',
 'train_time/model': '5.24s'}
Final reward: 200.0, length: 200.0


C:\Users\Lenovo\anaconda3\lib\site-packages\tianshou\data\collector.py:61: UserWarning: Single environment detected, wrap to DummyVectorEnv.
  warnings.warn("Single environment detected, wrap to DummyVectorEnv.")


In [14]:
policy.load_state_dict(torch.load('dqn.pth'))

<All keys matched successfully>

In [15]:
policy.eval()
policy.set_eps(0.05)
collector = ts.data.Collector(policy, env, exploration_noise=True)
collector.collect(n_episode=1, render=1 / 35)

C:\Users\Lenovo\anaconda3\lib\site-packages\tianshou\data\collector.py:61: UserWarning: Single environment detected, wrap to DummyVectorEnv.
  warnings.warn("Single environment detected, wrap to DummyVectorEnv.")


{'n/ep': 1,
 'n/st': 185,
 'rews': array([185.]),
 'lens': array([185]),
 'idxs': array([0]),
 'rew': 185.0,
 'len': 185.0,
 'rew_std': 0.0,
 'len_std': 0.0}